<a href="https://colab.research.google.com/github/ESGcompetition/ipnyb/blob/choiyebin/%EA%B3%B5%EB%AA%A8%EC%A0%84_%EA%B0%80%EC%A4%91%EC%B9%98%ED%95%A8%EC%88%98_%EB%AA%A8%EB%93%A0%EC%97%B0%EB%8F%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [1]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')

my_path = '/content/package'
save_path = '/content/drive/MyDrive/Colab Notebooks/package' 

os.symlink(save_path, my_path)
sys.path.insert(0, my_path)

Mounted at /content/drive


In [117]:
import pandas as pd
import numpy as np
import re
import math

# 데이터셋 생성

## JSON 데이터

In [3]:
import requests, json
import pandas as pd

NOTION_TOKEN = "secret_AIc5tovrF5rnoYWC54z9CzwQUNb5d2A1q3HPKwOEqup"
DATABASE_ID = "213b4ff8bb884cfc987139ae53aa4396"

headers = {
    "Authorization": "Bearer " + NOTION_TOKEN,
    "Content-Type": "application/json",
    "Notion-Version": "2022-06-28",
}

In [4]:
DATABASE_ID_LIST = ["db2b026a03f0442f828bc88dcdbf143c", "c238334399d54b10aad0bcb84cb77d5c", "bf7efa466c624f1ca200d05c5cf34f58", "2f0e230f82974dd09841ae871cb558f9", "b708c8576ea34bb385d24aedc5b1277b", "d6211c2113914ecc86332d4bb305a115", "a8b0289c72eb4daaa1063152eb5e787a", "73e5dc61f97f4f2095c0265f2dae70d8", "1a91276509b246dda4c170a248d096c2", "6a374753947e42f7aa57fac3a474b78f"]
YEAR_LIST = [2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]

In [5]:
def readDatabase(DATABASE_ID_LIST, YEAR_LIST, headers):
    for id in zip(DATABASE_ID_LIST, YEAR_LIST):
        readUrl = f"https://api.notion.com/v1/databases/{id[0]}/query"
        res = requests.request("POST", readUrl, headers=headers)
        data = res.json()
        print(res.status_code)

        if res.status_code == 200:
            try:
                with open(f"./{id[1]}.json", 'w', encoding='utf8') as f:
                    json.dump(data, f, ensure_ascii=False)
            except:
                continue

In [6]:
readDatabase(DATABASE_ID_LIST, YEAR_LIST, headers)

200
200
200
200
200
200
200
200
200
200


In [7]:
def jsonToDataFrame(YEAR_LIST):
    temp = {}
    year_list = []
    name = []
    birth = []
    registered = []
    fullTime = []
    responsibilities = []
    career = []
    stock = []
    time1 = []
    time2 = []
    position = []

    for year in YEAR_LIST:
        with open(f"./{year}.json", 'r') as f:
            json_data = json.load(f)
            for i in range(len(json_data['results'])):
                properties = json_data['results'][i]['properties']
                year_list.append(year)
                for key in properties:
                    if key == '이름':
                        name.append(properties[key]['title'][0]['text']['content'])
                    elif key == '출생년월':
                        birth.append(properties[key]['rich_text'][0]['text']['content'])
                    elif key == '등기임원 여부':
                        registered.append(properties[key]['rich_text'][0]['text']['content'])
                    elif key == '상근 여부':
                        fullTime.append(properties[key]['rich_text'][0]['text']['content'])
                    elif key == '담당업무':
                        responsibilities.append(properties[key]['rich_text'][0]['text']['content'])
                    elif key == '주요경력':
                        career.append(properties[key]['rich_text'][0]['text']['content'])
                    elif key == '의결권 있는 주식수':
                        stock.append(properties[key]['number'])
                    elif key == '재직기간':
                        time1.append(properties[key]['rich_text'][0]['text']['content'])
                    elif key == '임기만료일':
                        time2.append(properties[key]['rich_text'][0]['text']['content'])
                    elif key == '직위':
                        position.append(properties[key]['rich_text'][0]['text']['content'])

    temp['이름'] = name
    temp['연도'] = year_list
    temp['출생년월'] = birth
    temp['등기임원 여부'] = registered
    temp['상근 여부'] = fullTime
    temp['담당업무'] = responsibilities
    temp['주요경력'] = career
    temp['의결권 있는 주식수'] = stock
    temp['재직기간'] = time1
    temp['임기만료일'] = time2
    temp['직위'] = position
    return temp

In [8]:
dict = jsonToDataFrame(YEAR_LIST)

In [9]:
df = pd.DataFrame.from_dict(data=dict, orient='columns')

In [10]:
df.head()

,이름,연도,출생년월,등기임원 여부,상근 여부,담당업무,주요경력,의결권 있는 주식수,재직기간,임기만료일,직위
0,이현식,2014,1962.05,미등기임원,상근,강북지역본부장,서울대 경영학(석사)\n대신증권 시흥동지점장\n대신증권 명일동지점장\n대신증권 퇴직...,4720.0,2012.04.01~2014.12.31,2015.03.20,전무
1,조윤남,2014,1968.11,미등기임원,상근,리서치센터장,한국과학기술원 화학공학(석사)\n대신증권 투자전략부장,17401.0,2011.04.01~2014.12.31,2015.03.20,상무
2,나재철,2014,1960.01,등기임원,상근,업무총괄,한국외국어대 경영학(석사)\n대신증권 강남지역본부장\n대신증권 WM추진본부장\n대...,23301.0,2004.04.01~2014.12.31,2016.03.31,대표이사
3,이정훈,2014,1947.06,등기임원,비상근,-,서울대 법학(석사)\n대한중재인협회 회장\n법무법인 태평양 대표변호사(현),0.0,2010.05.28~2014.12.31,2015.03.20,사외이사
4,송혁,2014,1964.04,미등기임원,상근,비서/브랜드담당,전북대 회계학\n대신증권 인수공모2부\n대신증권 비서실장,10391.0,2010.04.01~2014.12.31,2015.03.20,전무


## 데이터셋 살펴보기

In [11]:
print(len(df[df['연도']==2014]))
print(len(df[df['연도']==2015]))
print(len(df[df['연도']==2016]))
print(len(df[df['연도']==2017]))
print(len(df[df['연도']==2018]))
print(len(df[df['연도']==2019]))
print(len(df[df['연도']==2020]))
print(len(df[df['연도']==2021]))
print(len(df[df['연도']==2022]))
print(len(df[df['연도']==2023]))

30
31
35
32
33
37
36
38
42
40


In [12]:
df['담당업무'].unique()

array(['강북지역본부장', '리서치센터장', '업무총괄', '-', '비서/브랜드담당', '강남지역본부장',
       'Trading Center장', '서부지역본부장', '인재역량센터장', '동부지역본부장',
       'Wholesale사업단장', '영업부장', '고객자산본부장', '인프라관리본부장', '상근감사위원',
       '금융주치의사업단장', '스마트금융본부장', 'IB사업단장', '금융주치의추진본부장',
       '업무총괄(IB사업단, 고객자산본부 제외)', '울산지점장', '기획본부장', 'IT서비스본부장', '감사위원',
       '구조화상품본부장', '감사위원장', 'IB부문장', '정보보호담당', '감사담당', 'IB부부문장',
       'Wholesale영업본부장', 'Operation&Technology본부장',
       'Research&Strategy본부장', '리스크관리담당', '경영지원본부장', 'IB1부문장',
       'Market\n  Solution부문장', '기획본부장 / 투자금융담당', '준법감시인 / 준법지원담당',
       'Product부문장', '업무총괄(IB사업단,고객자산본부 제외)', 'IB2부문장', '고객자산본부장 / 홍보담당',
       'Advisory본부장', 'Solution&Product사업단장', '비서/브랜드본부',
       'Market Solution부문장', '강남선릉센터장', '스마트Biz본부장', '전략지원담당',
       '준법감시인\n준법지원담당', '고객자산본부장/ 홍보담당', 'PF부문장',
       '업무총괄 (IB사업단,고객자산본부 제외)', 'IT본부장', '경영기획본부장', 'WM사업단장', '경영전략총괄',
       '전략지원부문장/프라이빗라운지 부문장', 'Market Solution 부문장', 'Research Center장',
       '정보보호부문장', '고객자산본부장/ 홍보부문장',
       '업무

In [13]:
df['주요경력'].unique()

array(['서울대 경영학(석사)\n대신증권 시흥동지점장\n대신증권 명일동지점장\n대신증권 퇴직연금컨설팅본부장\n대신증권 퇴직연금본부장',
       '한국과학기술원 화학공학(석사)\n대신증권 투자전략부장',
       '한국외국어대 \xa0경영학(석사)\n대신증권 강남지역본부장\n대신증권 WM추진본부장\n대신증권 Wholesale본부장\n대신증권 Wholesale사업단장\n기획본부장대신증권 \xa0인재역량센터장\n기업금융사업단장',
       '서울대 법학(석사)\n대한중재인협회 회장\n법무법인 태평양 대표변호사(현)',
       '전북대 회계학\n대신증권 인수공모2부\n대신증권 비서실장',
       '성균관대 불어불문학\n대신증권 대치동지점 이사대우지점장\n대신증권 남대문지점 이사대우지점장\n대신증권 종로지점 이사대우지점장\n대신증권 강남지역본부장\n대신증권\xa0중부지역본부장',
       '조지아공대 항공우주학(박사)\n대신증권 파생상품운용부 부장\n대신증권 자산운용본부 이사대우본부장',
       '고려대 법학\n대신증권 화정동지점장\n대신증권 상무지점장\n대신증권 무등지점 이사대우지점장',
       '서강대 경제학\n대신증권 영업기획부 부장\n대신증권 기획실 실장\n대신증권 인재전략부 이사대우부장',
       '고려대 경영대학원(석사)\n금융감독원 부원장보\nLIG손해보험 상근감사위원\n하비스트 대표(현)',
       '성균관대 회계학\n대신증권 무역센터지점장\n대신증권\xa0분당지점장\n대신증권 강남선릉센터 이사대우센터장\n대신증권 강서지역본부장',
       '한국외국어대 무역학(석사)\n우리투자증권 기업분석팀장\n대신증권 리서치센터장\n대신증권 기획본부장',
       '연세대 경영학\n대신증권 법인영업부장\n대신증권 Wholesale1본부장\n대신증권 Wholesale영업본부장',
       '연세대 경제학(석사)\n우리CS자산운용 AI본부장\n대신증권 파생금융본부장\n대신증권 CM본부장\n대신증권\xa0Ca

# 데이터 전처리

In [14]:
#의결권 있는 주식수 Nan 전부 0처리
df['의결권 있는 주식수'].isnull().sum()

11

In [15]:
df['의결권 있는 주식수'].fillna(0.0, inplace=True)

In [16]:
df['의결권 있는 주식수'].isnull().sum()

0

In [17]:
#'\xa0' 처리
df['주요경력']=df['주요경력'].str.replace('\xa0', '')

In [18]:
df['주요경력'].unique()

array(['서울대 경영학(석사)\n대신증권 시흥동지점장\n대신증권 명일동지점장\n대신증권 퇴직연금컨설팅본부장\n대신증권 퇴직연금본부장',
       '한국과학기술원 화학공학(석사)\n대신증권 투자전략부장',
       '한국외국어대 경영학(석사)\n대신증권 강남지역본부장\n대신증권 WM추진본부장\n대신증권 Wholesale본부장\n대신증권 Wholesale사업단장\n기획본부장대신증권 인재역량센터장\n기업금융사업단장',
       '서울대 법학(석사)\n대한중재인협회 회장\n법무법인 태평양 대표변호사(현)',
       '전북대 회계학\n대신증권 인수공모2부\n대신증권 비서실장',
       '성균관대 불어불문학\n대신증권 대치동지점 이사대우지점장\n대신증권 남대문지점 이사대우지점장\n대신증권 종로지점 이사대우지점장\n대신증권 강남지역본부장\n대신증권중부지역본부장',
       '조지아공대 항공우주학(박사)\n대신증권 파생상품운용부 부장\n대신증권 자산운용본부 이사대우본부장',
       '고려대 법학\n대신증권 화정동지점장\n대신증권 상무지점장\n대신증권 무등지점 이사대우지점장',
       '서강대 경제학\n대신증권 영업기획부 부장\n대신증권 기획실 실장\n대신증권 인재전략부 이사대우부장',
       '고려대 경영대학원(석사)\n금융감독원 부원장보\nLIG손해보험 상근감사위원\n하비스트 대표(현)',
       '성균관대 회계학\n대신증권 무역센터지점장\n대신증권분당지점장\n대신증권 강남선릉센터 이사대우센터장\n대신증권 강서지역본부장',
       '한국외국어대 무역학(석사)\n우리투자증권 기업분석팀장\n대신증권 리서치센터장\n대신증권 기획본부장',
       '연세대 경영학\n대신증권 법인영업부장\n대신증권 Wholesale1본부장\n대신증권 Wholesale영업본부장',
       '연세대 경제학(석사)\n우리CS자산운용 AI본부장\n대신증권 파생금융본부장\n대신증권 CM본부장\n대신증권Capital Market 사업단장',


In [19]:
#대학, 전공 피처 생성
temp=df['주요경력'].str.split('\n')
colleges=temp.str.get(0)

#추가로 처리해줘야될 대학 form 맞춰주기
colleges[colleges== 'University of Bath 사회과학대학원(석사)'] ='UniversityofBath 사회과학대학원(석사)'
colleges[colleges== 'Florida International University 경영대학원(박사)']='FloridaInternationalUniversity 경영대학원(박사)'
colleges[colleges== 'Ohio State University 정책대학원(박사)'] = 'OhioStateUniversity 정책대학원(박사)'
colleges[colleges== 'KDI국제정책대학원(석사)'] = 'KDI 국제정책대학원(석사)'
colleges[colleges== '상명여자사범대'] = '상명여자사범대 교육학'
colleges[colleges== '한국외국어대경영학(석사)'] = '한국외국어대 경영학(석사)'
colleges[colleges== '미시간대학교 경영대학원 MBA'] = '미시간대학교 경영대학원MBA'

#대학, 전공으로 나누기
ttemp = colleges.str.split() 

#대학 피처 생성
df['대학교']=ttemp.str.get(0)

#전공 피처 생성
tttemp = ttemp.str.get(1)
tttemp=tttemp.replace(r'\([^)]*\)', '', regex=True)
df['전공']=tttemp

In [20]:
df['대학교'].unique()

array(['서울대', '한국과학기술원', '한국외국어대', '전북대', '성균관대', '조지아공대', '고려대', '서강대',
       '연세대', '대전대', '한양대', 'KDI', '상명여자사범대', '울산대', '홍익대', '중앙대', '명지대',
       'Brown대', '전남대', '강원대', '부경대', '영남공업전문대', '경북대', '동국대', '카네기멜론대',
       '경희대', '충북대', '조선대', '세종대', 'UniversityofBath', '가천대학교',
       'OhioStateUniversity', '충북대학교', '동아대', '세종대학교', '한국외대', '성균관대학교',
       'FloridaInternationalUniversity', '건국대', '원광대', '인하대', '연세대학교',
       '미시간대학교'], dtype=object)

In [21]:
df['전공'].unique()

array(['경영학', '화학공학', '법학', '회계학', '불어불문학', '항공우주학', '경제학', '경영대학원',
       '무역학', '정치학', '국제정책대학원', '교육학', '전자계산학', '세무학', '수학', '전자전산학',
       '사법학', '국제경영학', '금융공학', '행정학', '산업정보학', '인적자원개발학', '전산학', 'MBA',
       '사회과학대학원', '정책대학원', '기계공학', '사회학', '통계학', '응용통계학', '정치외교학',
       '경제통상학', '국제금융학', '신문방송학', '경영대학원MBA'], dtype=object)

In [22]:
df['재직기간'].unique()

array(['2012.04.01~2014.12.31', '2011.04.01~2014.12.31',
       '2004.04.01~2014.12.31', '2010.05.28~2014.12.31',
       '2010.04.01~2014.12.31', '2013.04.01~2014.12.31',
       '2014.01.01~2014.12.31', '2013.06.07~2014.12.31',
       '2007.05.02~2014.12.31', '2007.06.22~2014.12.31',
       '2011.05.27~2014.12.31', '2014.01.16~2014.12.31',
       '2009.04.01~2014.12.31', '2004.09.24~2014.12.31',
       '2007.10.01~2014.12.31', '2007.04.01~2014.12.31',
       '2011.04.01~2015.12.31', '2015.03.20~2015.12.31',
       '2013.06.07~2015.12.31', '2014.01.16~2015.12.31',
       '2007.06.22~2015.12.31', '2007.04.01~2015.12.31',
       '2007.10.01~2015.12.31', '2013.04.01~2015.12.31',
       '2014.01.01~2015.12.31', '2004.04.01~2015.12.31',
       '2015.01.01~2015.12.31', '2004.09.24~2015.12.31',
       '2015.03.09~2015.12.31', '2010.04.01~2015.12.31',
       '2012.04.01~2015.12.31', '2011.05.27~2015.12.31',
       '2015.01.01~2016.12.31', '2010.04.01~2016.12.31',
       '2016.01.01~2016.12.31',

In [23]:
#근속연수 생성
def create_work_years(x):
  temp = re.split(r'~|~\n', x)
  temp[0] = temp[0].strip()
  temp[1] = temp[1].strip()
  return int(temp[1][:4]) - int(temp[0][:4])

df['근속연수'] = df['재직기간'].apply(lambda x : create_work_years(x))

In [24]:
df['근속연수'].unique()

array([ 2,  3, 10,  4,  1,  0,  7,  5,  8, 11,  6, 12,  9, 13, 14, 15, 16,
       17, 18, 19])

In [27]:
'''
#주요경력 전처리
def replace_career(x):
  temp = [string.split() for string in re.split(r'\n |\n', x)]
  if len(temp)==1 :
    return temp
  else:
    return set(list(zip(*temp[1:]))[0])

df['주요경력요약'] = df['주요경력'].apply(lambda x : replace_career(x))
'''

#주요경력에 하나만 있어서 안나눠지는거 확인.
#전부 한사람 이어룡씨(대신증권 회장)이어서 집합을 {대신증권으로 바꿈.}
def classify_value(value):
    if isinstance(value, set):
        return False
    elif isinstance(value, list):
        return True
    else:
        return 'Unknown'

temp = df['주요경력요약'].apply(classify_value)
df[temp]

,이름,연도,출생년월,등기임원 여부,상근 여부,담당업무,주요경력,의결권 있는 주식수,재직기간,임기만료일,직위,대학교,전공,근속연수,주요경력요약


In [26]:
#주요경력 전처리
def replace_career(x):
  temp = [string.split() for string in re.split(r'\n |\n', x)]
  if len(temp)==1 :
    return {'대신증권'}
  else:
    return set(list(zip(*temp[1:]))[0])

df['주요경력요약'] = df['주요경력'].apply(lambda x : replace_career(x))

In [28]:
df['주요경력요약'].head(50)

0                                   {대신증권}
1                                   {대신증권}
2              {기업금융사업단장, 기획본부장대신증권, 대신증권}
3                          {법무법인, 대한중재인협회}
4                                   {대신증권}
5                      {대신증권중부지역본부장, 대신증권}
6                                   {대신증권}
7                                   {대신증권}
8                                   {대신증권}
9                   {하비스트, LIG손해보험, 금융감독원}
10                       {대신증권분당지점장, 대신증권}
11                          {우리투자증권, 대신증권}
12                                  {대신증권}
13           {대신증권Capital, 우리CS자산운용, 대신증권}
14                                  {대신증권}
15                 {푸르덴셜투자증권, 메릴린치증권, SBC}
16                                  {대신증권}
17                          {한국투자증권, 부영주택}
18                         {IBK투자증권, 대신증권}
19                      {동양증권, 삼성증권, 대신증권}
20                                  {대신증권}
21                        {대신투자신탁운용, 대신증권}
22                      {대신증권, 대신증권무거동지점장}
23         

In [29]:
df.head()

,이름,연도,출생년월,등기임원 여부,상근 여부,담당업무,주요경력,의결권 있는 주식수,재직기간,임기만료일,직위,대학교,전공,근속연수,주요경력요약
0,이현식,2014,1962.05,미등기임원,상근,강북지역본부장,서울대 경영학(석사)\n대신증권 시흥동지점장\n대신증권 명일동지점장\n대신증권 퇴직...,4720.0,2012.04.01~2014.12.31,2015.03.20,전무,서울대,경영학,2,{대신증권}
1,조윤남,2014,1968.11,미등기임원,상근,리서치센터장,한국과학기술원 화학공학(석사)\n대신증권 투자전략부장,17401.0,2011.04.01~2014.12.31,2015.03.20,상무,한국과학기술원,화학공학,3,{대신증권}
2,나재철,2014,1960.01,등기임원,상근,업무총괄,한국외국어대 경영학(석사)\n대신증권 강남지역본부장\n대신증권 WM추진본부장\n대신...,23301.0,2004.04.01~2014.12.31,2016.03.31,대표이사,한국외국어대,경영학,10,"{기업금융사업단장, 기획본부장대신증권, 대신증권}"
3,이정훈,2014,1947.06,등기임원,비상근,-,서울대 법학(석사)\n대한중재인협회 회장\n법무법인 태평양 대표변호사(현),0.0,2010.05.28~2014.12.31,2015.03.20,사외이사,서울대,법학,4,"{법무법인, 대한중재인협회}"
4,송혁,2014,1964.04,미등기임원,상근,비서/브랜드담당,전북대 회계학\n대신증권 인수공모2부\n대신증권 비서실장,10391.0,2010.04.01~2014.12.31,2015.03.20,전무,전북대,회계학,4,{대신증권}


# 데이터셋 연도별 분리

In [30]:
df_2014 = df[df['연도']==2014].reset_index(drop=True)
df_2015 = df[df['연도']==2015].reset_index(drop=True)
df_2016 = df[df['연도']==2016].reset_index(drop=True)
df_2017 = df[df['연도']==2017].reset_index(drop=True)
df_2018 = df[df['연도']==2018].reset_index(drop=True)
df_2019 = df[df['연도']==2019].reset_index(drop=True)
df_2020 = df[df['연도']==2020].reset_index(drop=True)
df_2021 = df[df['연도']==2021].reset_index(drop=True)
df_2022 = df[df['연도']==2022].reset_index(drop=True)
df_2023 = df[df['연도']==2023].reset_index(drop=True)

In [31]:
df_2022

,이름,연도,출생년월,등기임원 여부,상근 여부,담당업무,주요경력,의결권 있는 주식수,재직기간,임기만료일,직위,대학교,전공,근속연수,주요경력요약
0,박정환,2022,1975.07,미등기,상근,Trading부문장,서울대 기계공학\nSC제일은행 외환파생상품 이사대우\n미래에셋증권 FICC파생운용팀...,0.0,2021.12.27~2022.12.31,2024.3.31,상무,서울대,기계공학,1,"{KTB투자증권, SC제일은행, 미래에셋증권}"
1,강준규,2022,1972.02,미등기,상근,디지털부문장,가천대학교 무역학\n대신증권 수지지점장\n대신증권 여의도영업부장\n재경1WM부문장,4349.0,2020.11.23~2022.12.31,2024.3.31,상무,가천대학교,무역학,2,"{재경1WM부문장, 대신증권}"
2,최근영,2022,1963.12,미등기,상근,준법감시인\n준법지원부문장,조선대 회계학\n대신자산운용 법무본부장\n대신자산운용 리스크관리본부장/준법감시인대신...,10437.0,2020.01.21~2022.12.31,2023.12.31,상무,조선대,회계학,2,"{대신저축은행, 대신자산운용, 대신증권}"
3,이순남,2022,1969.05,미등기,상근,프라이빗부문장\n나인원프라이빗라운지장,동국대 경영학(석사)\n대신증권 강남역지점장\n대신증권 강남역삼센터 이사대우센터장대...,10953.0,2018.01.01~2022.12.31,2024.3.31,상무,동국대,경영학,4,{대신증권}
4,김병철,2022,1958.09,사외이사,비상근,감사위원장,성균관대 행정학(박사)\n감사원 감사위원\n現)금융위원회 적극행정심의위원,0.0,2018.03.23~2022.12.31,2023.3.31,사외이사,성균관대,행정학,4,"{現)금융위원회, 감사원}"
5,이정화,2022,1968.11,미등기,상근,-,부경대 무역학\n대신증권 마산지점장\n대신증권 부전동지점장\n대신증권 동래지점장,9442.0,2016.01.01~2022.12.31,2023.3.31,전무,부경대,무역학,6,{대신증권}
6,최광철,2022,1973.12,미등기,상근,WM추진부문장,서강대 사회학\n대신증권 상품전략부장\n대신증권 상품기획부장\n대신증권 리테일기획부장,0.0,2022.11.21~2022.12.31,2024.3.31,상무,서강대,사회학,0,{대신증권}
7,이재우,2022,1972.08,미등기,상근,Wholesale부문장,홍익대 경영학\n대신증권 재무관리부장/자금부장\n대신증권 재무자금부장\n대신증권 법...,22924.0,2019.11.25~2022.12.31,2024.3.31,전무,홍익대,경영학,3,{대신증권}
8,길기모,2022,1968.12,미등기,상근,리스크관리부문장,서울대 경제학\n메리츠종금증권 심사팀장\n메리츠종금증권 리스크관리본부장,3532.0,2019.04.01~2022.12.31,2023.3.31,전무,서울대,경제학,3,{메리츠종금증권}
9,김성호,2022,1950.03,사외이사,비상근,감사위원,건국대 법학(박사)\n국가청렴위원회 사무처장\n제58대 법무부장관\n現)재단법인 행...,0.0,2022.03.18~2022.12.31,2024.3.31,사외이사,건국대,법학,0,"{국가청렴위원회, 제58대, 現)재단법인}"


# 가중치 함수

## 대학교

In [33]:
#학교 거리 계산
def cal_college(df1, df2, alpha = 0.7):
  #지역별로 나눔
  college_dict = {'서울경기':['상명여자사범대', '서울대', '홍익대', '성균관대', '연세대', '연세대학교', '동국대', '중앙대', '한국외대', '한국외국어대', '세종대학교', '세종대', '성균관대학교', '경희대', '서강대', '고려대', '건국대', '가천대', '가천대학교', '한양대', '명지대'],
                   '광역시':['부경대', '경북대', '전남대', '조선대', '한국과학기술원', '가천대', '인하대', '동아대', '대전대', '울산대', '영남공업전문대'],
                   '그외한국':['전북대', '원광대', '충북대', '충북대학교', '강원대', 'KDI'],
                   '해외':['FloridaInternationalUniversity', 'OhioStateUniversity', 'UniversityofBath', '조지아공대', 'Brown대', '카네기멜론대', '미시간대학교']}

  #키값 찾기
  for key, val in college_dict.items() :
    if df1['대학교'] in val :
      df1_key = key
    if df2['대학교'] in val :
      df2_key = key
          
  #k값 구하기
  if df1['대학교'] == df2['대학교'] :
    k=0
  elif df1_key == df2_key :
    k=1
  elif df1_key != df2_key :
    k=2

  return alpha ** k

(시별로 나누면 같은 지역이 너무 많고 지역마다 학교도 적어서 일단 아래처럼 나눔) \
1. 경기=>서울통합 \
2. 광역시통합 \
3. 광역시 제외 남은 도? (기타 한국) \
4. 해외

## 학과

In [34]:
df['전공'].unique()

array(['경영학', '화학공학', '법학', '회계학', '불어불문학', '항공우주학', '경제학', '경영대학원',
       '무역학', '정치학', '국제정책대학원', '교육학', '전자계산학', '세무학', '수학', '전자전산학',
       '사법학', '국제경영학', '금융공학', '행정학', '산업정보학', '인적자원개발학', '전산학', 'MBA',
       '사회과학대학원', '정책대학원', '기계공학', '사회학', '통계학', '응용통계학', '정치외교학',
       '경제통상학', '국제금융학', '신문방송학', '경영대학원MBA'], dtype=object)

In [35]:
#학과 거리 계산
def cal_department(df1, df2, alpha = 0.7):
  department_dict={'경영경제':['회계학', '세무학', '경제학', '국제경영학', '경영학', '무역학', 'MBA', '경제통상학', '경영대학원', '경영대학원MBA', '산업정보학', '금융공학', '국제금융학'],
                   '사회과학':['사회학', '사회과학대학원', '행정학', '정치외교학', '정치학', '신문방송학'],
                   '법률':['법학', '사법학', '정책대학원', '국제정책대학원'],
                   '인문':['불어불문학'],
                   '기타':['교육학', '인적자원개발학'],
                   '자연':['응용통계학', '통계학', '수학'],
                   '공학':['기계공학', '화학공학', '전자계산학', '전자전산학', '전산학', '항공우주학']}

  #키값 찾기
  for key, val in department_dict.items() :
    if df1['전공'] in val :
      df1_key = key
    if df2['전공'] in val :
      df2_key = key
          
  #k값 구하기
  if df1['전공'] == df2['전공'] :
    k=0
  elif df1_key == df2_key :
    k=1
  elif df1_key != df2_key :
    k=2

  return alpha ** k

## 직위

In [36]:
df['직위'].unique()

array(['전무', '상무', '대표이사', '사외이사', '상무보', '감사위원', '회장', '사장', '부사장',
       '이사대우담당', '이사대우부문장', '부회장'], dtype=object)

In [37]:
#직위 거리 계산
def cal_position(df1, df2, alpha = 0.7):
  position_dict={'회장':0, '대표이사':1, '부회장':2, '사장':3, '부사장':4, '전무':5, '상무': 6, '상무보':7, '이사대우부문장':8, '이사대우담당':8, '사외이사':99, '감사위원':99}

  #value값 찾기
  df1_value = position_dict[df1['직위']]
  df2_value = position_dict[df2['직위']]

  #k값 구하기
  if df1['직위'] == df2['직위'] :
    k=0
  elif df1_value == 99 or df2_value == 99 : #사외이사 혹은 감사인원(회사밖 인물)은 모두와 거리 4(최대차이의 중간값// 회장, 이사대우담당과의 거리)
    k=4
  else :
    k=abs(df1_value - df2_value) #값 차이만큼 거리

  return alpha ** k

대신증권 계급도 못찾겠음..

## 근속연수

In [38]:
# 근속연수 거리 계산
def cal_work_years(df1, df2, alpha = 0.7):
  #k값 구하기
  if df1['근속연수'] == df2['근속연수'] :
    k=0
  else :
    if abs(df1['근속연수'] - df2['근속연수']) % 5 ==0:    #근속연수 차이 5년 기준으로 1씩 증가 (차이 1~5/6~10/11~15/...같은그룹)
      k= abs(df1['근속연수'] - df2['근속연수']) // 5 
    else:
      k= 1 + abs(df1['근속연수'] - df2['근속연수']) // 5 

  return alpha ** k  

## 담당업무

In [39]:
df['담당업무'].unique()

array(['강북지역본부장', '리서치센터장', '업무총괄', '-', '비서/브랜드담당', '강남지역본부장',
       'Trading Center장', '서부지역본부장', '인재역량센터장', '동부지역본부장',
       'Wholesale사업단장', '영업부장', '고객자산본부장', '인프라관리본부장', '상근감사위원',
       '금융주치의사업단장', '스마트금융본부장', 'IB사업단장', '금융주치의추진본부장',
       '업무총괄(IB사업단, 고객자산본부 제외)', '울산지점장', '기획본부장', 'IT서비스본부장', '감사위원',
       '구조화상품본부장', '감사위원장', 'IB부문장', '정보보호담당', '감사담당', 'IB부부문장',
       'Wholesale영업본부장', 'Operation&Technology본부장',
       'Research&Strategy본부장', '리스크관리담당', '경영지원본부장', 'IB1부문장',
       'Market\n  Solution부문장', '기획본부장 / 투자금융담당', '준법감시인 / 준법지원담당',
       'Product부문장', '업무총괄(IB사업단,고객자산본부 제외)', 'IB2부문장', '고객자산본부장 / 홍보담당',
       'Advisory본부장', 'Solution&Product사업단장', '비서/브랜드본부',
       'Market Solution부문장', '강남선릉센터장', '스마트Biz본부장', '전략지원담당',
       '준법감시인\n준법지원담당', '고객자산본부장/ 홍보담당', 'PF부문장',
       '업무총괄 (IB사업단,고객자산본부 제외)', 'IT본부장', '경영기획본부장', 'WM사업단장', '경영전략총괄',
       '전략지원부문장/프라이빗라운지 부문장', 'Market Solution 부문장', 'Research Center장',
       '정보보호부문장', '고객자산본부장/ 홍보부문장',
       '업무

In [111]:
#담당업무 거리 계산
def cal_task(df1, df2, alpha = 0.7):
  task_dict={'IT계열': {'IT':['IT부문장', 'IT본부장', 'IT서비스본부장', '디지털부문장', '스마트Biz본부장', 'Operation&Technology본부장'], '정보보호':['정보보호부문장', '정보보호담당']},
             '본사영업':{'Wolesale':['Wholesale부문장','Wholesale영업본부장', 'Wholesale사업단장', 'Trading부문장', 'Trading Center장', 'Sales&Trading총괄', '대외협력담당'], 
                     'IB':['IB사업단장', 'IB부문장', 'IB부부문장', 'IB1부문장', 'IB2부문장', 'IPO담당',  'PF부문장', 'PF1본부장', '구조화상품본부장'], 
                     '영업':['영업부장']},
             '본사관리':{'리테일':['리테일총괄', 'Club1962센터장','WM추진부문장', 'WM추진본부장', 'WM사업단장'],
                     '리스크':['리스크관리부문장', '리스크관리담당'] ,
                     '경영':['경영전략총괄', '전략지원부문장', '전략지원담당', '경영지원본부장', '경영지원부문장', '경영기획부문장', '경영기획본부장', '기획본부장', '기획본부장 / 투자금융담당', '전략지원부문장/프라이빗라운지 부문장'], 
                     '상품':['Product부문장', 'Solution&Product사업단장'],
                     '업무':['업무총괄',  '업무총괄(IB사업단, 고객자산본부 제외)', '업무총괄(IB사업단,고객자산본부 제외)', '업무총괄 (IB사업단,고객자산본부 제외)', '업무총괄(IB사업단, 고객자산본부, 경영지원본부, 정보보호부문, 준법지원부문, 감사부문 제외)'], 
                     '금융':['기업금융담당', '스마트금융본부장', '금융주치의추진본부장', '금융주치의사업단장'], 
                     '인사,인프라':['인재역량센터장', '인프라관리본부장'],
                     '법률':['준법감시인\n 준법지원부문장', '준법감시인 준법지원부문장', '준법감시인\n준법지원부문장', '준법감시인\n준법지원담당', '준법감시인/준법지원부문장', '준법감시인 / 준법지원담당', 'Advisory본부장'], 
                     '감사':['감사위원장', '감사부문장', '감사위원', '상근감사위원', '감사담당'], 
                     '언론홍보':['홍보부문장', 'Coverage본부장']},
             '리서치': {'리서치센터':['Research Center장', '리서치센터장', 'Market\n  Solution부문장', 'Research&Strategy본부장', 'Market Solution부문장', 'Market Solution 부문장']},
             '고객':{'고객자산':['고객자산부문장', '고객자산부문 부부문장', '고객자산본부장', '고객자산본부장 / 홍보담당', '고객자산본부장/ 홍보담당', '고객자산본부장/ 홍보부문장', '고객자산본부장/홍보부문장', '고객자산부문장, 홍보부문장', '고객자산부문장\n홍보부문장'],
                   'WM':['서부WM부문장', '서부WM본부장', '재경1WM부문장', '재경1WM본부장', '재경2WM본부장', '재경2WM부문장', '동부WM부문장', '동부WM본부장'],
                   '프라이빗':['프라이빗부문장,\n 나인원프라이빗라운지장', '프라이빗부문장, 나인원프라이빗라운지장', '대신나인원 \n프라이빗라운지장', '프라이빗라운지 부문장 /대신나인원 프라이빗라운지장', '프라이빗부문장, 대신나인원프라이빗라운지장', '프라이빗부문장\n나인원프라이빗라운지장',], 
                   '소비자보호':['금융소비자보호부문장\n(COO)', '금융소비자보호부문장(COO)', '금융소비자보호부문장 (COO)', '금융소비자보호 총괄', '금융소비자보호총괄']},
             '기타':{'지점':['강북지역본부장', '강남지역본부장', '강남선릉센터장', '서부지역본부장', '동부지역본부장', '울산지점장'],
                   '비서':['비서실장', '비서/브랜드본부', '비서/브랜드담당'], '공란':['-']}}

   #키값 찾기
  for outerKey in task_dict.keys() :
    for key, val in task_dict[outerKey].items() :
      if df1['담당업무'] in val :
        df1_outerKey = outerKey
        df1_innerKey = key
      if df2['담당업무'] in val :
        df2_outerKey = outerKey
        df2_innerKey = key
          
          
  #k값 구하기
  if df1['담당업무'] == df2['담당업무'] : #같은업무
    k=0
  elif df1_outerKey == df2_outerKey and df1_innerKey == df2_innerKey: #상위하위 혹은 비슷한업무
    k=1
  elif df1_outerKey == df2_outerKey : #같은계열업무
    k=2
  elif df1_outerKey != df2_outerKey :
    k=3 #다른계열업무
  return alpha ** k

http://money2.daishin.co.kr/company/recruit/job_introduce.shtml

http://www.educe.co.kr/company/jobinfo_view.php?sma_no=60&keyword=&search_txt2=

겸직은 그냥 앞에꺼에 넣음(대체적으로 앞에꺼가 좀 더 중요한 업무 같아서)

## 주요경력

In [202]:
#경력 거리 계산
def cal_career(df1, df2, alpha = 0.7):
  career_dict={1:['대신증권', '대신증권WM사업단장', 'WM추진부문장', '재경1WM부문장', '대신증권중부지역본부장', '기획본부장대신증권', '대신증권분당지점장', '대신증권무거동지점장', '대신증권Capital', 
                  '이사대우부본부장', '기획본부장대신증권인재역량센터장', '대신증권인재전략부', '대신증권파생상품운용부장', '대신증권자산운용본부', '대신증권영업기획부장', '대신증권기획실장', 
                  '대신증권홍보실', '대신증권브랜드전략실', '대신증권IT개발부장', '대신증권트레이딩시스템부장', '대신증권Global사업본부', '신증권', '기업금융사업단장'], #대신증권
               0:['대신에프앤아이', '대신투자신탁운용', '대신투자신탁운용상무이사', '대신에이엠씨', '대신자산운용', '대신저축은행'], #대신증권 자회사
               2:['메리츠종금증권', '메리츠증권', '미래에셋증권', 'KTB투자증권', 'SC제일은행', '하나금융투자', 'IBK투자증권', 'DB금융투자', 'NH투자증권', 'KB증권', 'LIG손해보험', '굿모닝신한증권', '메릴린치증권', 
                  '삼성증권', '동양증권', '한국투자증권', '한국투자증권평촌지점', '대우증권', '대우본부장', '대우증권전략기획본부', '대우증권해외사업부문', '대우증권IB사업부문', 
                  '우리투자증권', '우리CS자산운용', '우리투자증권기업금융2팀장', '우리프라이빗에쿼티', '푸르덴셜투자증권', '하이투자증권주식인수팀',  '하이투자증권주식인수팀', 'SBC', 'Bank', '도이치은행'], #타증권사
               3:['금융감독원', '금융위원회', '現)금융위원회', '감사원', '대검찰청', '법무법인', '現)법무법인', '세무법인', '국세청', '중부지방국세청', '서울북부지방검찰청','서울지방국세청장', '서울지방국세청', 
                  '제58대', '제22대', '기획재정부', '국세청,관세청,산업통상자원부', '자본시장연구원', '한국조세연구원', '국가청렴위원회', '대한중재인협회', '한국회계정보학회', 
                  '금융위원회금융발전심의위원(현)', '금융위원회적극행정심의위원(現)', '안진회계법인',  'L&C세무회계사무소', '변호사정상명법률사무소', '피앤비세무컨설팅', '現)법무법인(유)'], #법,정책
               4:['중앙대', '중앙대학교', '現)중앙대', '연세대학교', 'KAIST', '수원대', '수원대학교', '서울시립대', '現)서울시립대', '학교법인', '現)학교법인', '재단법인', '現)재단법인'], #학교, 교수, 재단
               5:['하비스트', '한국물산', '우리선물', '부영주택', 'LG경제연구원', 'Arthur'] #기타
               }
  
  #경력 여부 빈리스트 생성
  df1_list = [0,0,0,0,0,0] #0번인덱스: 대신증권, 1번인덱스: 대신증권 자회사, 2번인덱스: 타증권사, 3번인덱스: 기타
  df2_list = [0,0,0,0,0,0]

  #리스트에 각 업종에 종사했었으면 1, 아니면 0
  for career in df1['주요경력요약'] :
    for key, val in career_dict.items() :
      if career in val :
        df1_list[key]=1

  for career in df2['주요경력요약'] :
    for key, val in career_dict.items() :
      if career in val :
        df2_list[key]=1

  #df1_list 와 df2_list 합 구하기(요소가 0-> 둘다 종사 안했음, 1-> 한쪽만 종사함, 2-> 둘다 종사함)
  sum_list = [x + y for x, y in zip(df1_list, df2_list)]

  #k값 구하기
  if df1_list == df2_list : #전체 같으면 0
    k=0

  elif (sum_list[0]==0 and sum_list[1]==0 and sum_list[2]==0): #기타만 있을 때 (법률, 학교, 기타) 
    count2 = sum_list.count(2) #2인 요소 개수 찾기(df1, df2 같은거 개수)
    k = 0.6 * (1 + count2) 

  else : #기타만 있는거 아니고, 
    #법률, 학교, 기타 전부 기타로 통합. 범주는 3으로.
    if sum(df1_list[3:]) > 0 :
      df1_list[3] = 1
      df1_list[4] = 0
      df1_list[5] = 0
    if sum(df2_list[3:]) > 0 :
      df2_list[3] = 1
      df2_list[4] = 0
      df2_list[5] = 0

    #수정한 df1_list, df2_list로 sum_list 다시 생성
    sum_list = [x + y for x, y in zip(df1_list, df2_list)]

    #df1_list, df2_list에서 1인 인덱스 뽑기
    index1 = [i for i in range(len(df1_list)) if df1_list[i] == 1]
    index2 = [i for i in range(len(df2_list)) if df2_list[i] == 1]

    min = 10
    max = 0

    for i in index1 :
      for j in index2 :
        if i != j: #두 사람 간에, 다른 업종끼리 최대거리 최소 거리 구하기

          if (i==3 and j != 3) or (j==3 and i != 3) : #기타와 다른 업종
            max = 4 #기타는 모두와 거리 4 고정 (최대거리 구하기)
            temp = 4
          else : #최대거리 구하기
            temp = abs(i-j)
            if temp > max:
              max = temp
          if temp < min: #최소거리 구하기
            min = temp
    
    k=float((max+min)/2) #거리는 최대거리, 최소거리의 평균
    k= k- (0.5 * sum_list.count(2)) # 같은 업종 있으면, 있는 만큼 거리 -0.5
  
  return alpha ** k

0. 대신증권
1. 대신증권 자회사
2. 타증권사
3. 법, 정책
4. 학교, 교수, 재단
5. 기타 다른분야

## 주식수

In [126]:
#보유 주식수별 거리 계산
def cal_stock(df1, df2, alpha = 0.7):
  #k값 구하기
  if df1['의결권 있는 주식수'] == df2['의결권 있는 주식수'] :
    k=0
  else :
    k= abs(df1['의결권 있는 주식수'] - df2['의결권 있는 주식수'])//100 #차이로 보기=> 차이가 거리가 됨(너무 차이 크면 거리 멀어짐), 100개 단위로 끊어서 보기
    k=math.log10(1+k) #로그스케일
  return alpha ** k  

## 가중치함수 통합

In [206]:
#가중치 함수
def weight_sum(df1, df2):
  wt_college = cal_college(df1, df2)
  wt_department = cal_department(df1, df2)
  wt_position = cal_position(df1, df2)
  wt_task = cal_task(df1, df2)
  wt_work_years = cal_work_years(df1, df2)
  wt_career = cal_career(df1, df2)
  wt_stock = cal_stock(df1, df2)
  return wt_college + wt_department + wt_position + wt_task + wt_work_years + wt_career + wt_stock

## 기타 테스트

In [207]:
cal_college(df_2021.iloc[1], df_2021.iloc[2])

0.7

In [208]:
cal_department(df_2021.iloc[1], df_2021.iloc[2])

0.7

In [209]:
cal_position(df_2022.iloc[1], df_2022.iloc[2])

1.0

In [210]:
cal_work_years(df_2022.iloc[0], df_2022.iloc[3])

0.7

In [211]:
cal_task(df_2022.iloc[33], df_2022.iloc[35])

0.3429999999999999

In [212]:
pd.set_option('display.max_rows', None)
print(df['주요경력요약'])

0                                        {대신증권}
1                                        {대신증권}
2                   {기업금융사업단장, 기획본부장대신증권, 대신증권}
3                               {법무법인, 대한중재인협회}
4                                        {대신증권}
5                           {대신증권중부지역본부장, 대신증권}
6                                        {대신증권}
7                                        {대신증권}
8                                        {대신증권}
9                        {하비스트, LIG손해보험, 금융감독원}
10                            {대신증권분당지점장, 대신증권}
11                               {우리투자증권, 대신증권}
12                                       {대신증권}
13                {대신증권Capital, 우리CS자산운용, 대신증권}
14                                       {대신증권}
15                      {푸르덴셜투자증권, 메릴린치증권, SBC}
16                                       {대신증권}
17                               {한국투자증권, 부영주택}
18                              {IBK투자증권, 대신증권}
19                           {동양증권, 삼성증권, 대신증권}
20                                      

In [213]:
cal_career(df.iloc[2], df.iloc[48])

0.409963413001697

In [214]:
cal_stock(df_2022.iloc[37], df_2022.iloc[7])

0.40215966791391633

# 가중치 행렬

In [215]:
# 인물 가중치행렬
col = []
row = []
matrixs = []
dataFrames = [df_2014, df_2015, df_2016, df_2017, df_2018, df_2019, df_2020, df_2021, df_2022, df_2023]

for dataframe in dataFrames:
  for i in range(len(dataframe)):
      col.append(dataframe.loc[i, '이름'])
      row.append(dataframe.loc[i, '이름'])
  weight_matrix = pd.DataFrame(columns=col, index=row)
  matrixs.append(weight_matrix)
  col=[]
  row=[]

#matrixs[0]->2014년도 인물 가중치행렬
#matrixs[1]->2015년도 인물 가중치행렬
#matrixs[2]->2016년도 인물 가중치행렬
#matrixs[3]->2017년도 인물 가중치행렬
#matrixs[4]->2018년도 인물 가중치행렬
#....

In [216]:
matrixs[9]

,김봉진,김성호,양홍석,강준규,최근영,권택현,신재범,정회민,이순남,박석원,...,윤태림,이정훈,김수창,김상원,이어룡,홍종국,길기모,김범철,김성균,한승희
김봉진,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
김성호,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
양홍석,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
강준규,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
최근영,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
권택현,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
신재범,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
정회민,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
이순남,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
박석원,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [218]:
#가중치 계산
for t, dataframe in enumerate(dataFrames):
  for i in range(0, len(dataframe)):
    for j in range(0, len(dataframe)):
      matrixs[t].iloc[i, j] = weight_sum(dataframe.iloc[i], dataframe.iloc[j])

## 2014년

In [220]:
matrixs[0]

,이현식,조윤남,나재철,이정훈,송혁,하창룡,이동훈,박동현,홍대한,박찬수,...,이어룡,양홍석,김봉규,조경순,김병철,김창봉,박찬욱,배영훈,이인형,장우철
이현식,7.0,4.19519,4.21819,3.7092,4.914579,4.861587,4.092733,4.914002,4.649258,3.789064,...,3.588858,4.568487,4.970461,4.66199,4.731925,3.4722,3.9192,5.161228,3.2622,4.883848
조윤남,4.19519,7.0,3.511418,2.952513,4.239698,4.520211,4.616196,4.507891,4.573793,3.122376,...,3.182097,3.401603,4.424536,4.55473,4.499945,2.952513,2.952513,4.482228,3.252513,5.014473
나재철,4.21819,3.511418,7.0,2.93274,3.733585,3.651241,3.371974,3.644073,4.033667,3.312604,...,4.484759,4.625657,3.89758,4.025667,3.964636,3.28974,3.44274,4.268639,3.07974,3.941995
이정훈,3.7092,2.952513,2.93274,7.0,3.437232,3.363614,2.868746,3.890365,3.184817,5.176974,...,3.410747,3.211984,3.160445,3.192063,3.180408,4.884805,5.841805,3.121235,4.674805,3.029555
송혁,4.914579,4.239698,3.733585,3.437232,7.0,4.494002,4.098336,4.441587,4.544227,3.517096,...,3.379153,3.758539,4.466805,4.613507,4.604807,3.200232,3.647232,4.666057,3.200232,4.493474
하창룡,4.861587,4.520211,3.651241,3.363614,4.494002,7.0,4.396198,5.540848,4.787516,3.833478,...,3.538623,3.40152,5.119763,4.817587,4.263638,3.516614,3.363614,4.360309,3.006614,4.647002
이동훈,4.092733,4.616196,3.371974,2.868746,4.098336,4.396198,7.0,4.394628,4.700896,3.038609,...,3.18476,3.192054,4.095604,4.699494,4.101779,2.868746,2.868746,4.275757,3.078746,4.541244
박동현,4.914002,4.507891,3.644073,3.890365,4.441587,5.540848,4.394628,7.0,4.761992,4.150228,...,3.53854,3.401505,5.08,4.782,4.24199,3.533365,3.380365,4.356093,3.023365,4.802342
홍대한,4.649258,4.573793,4.033667,3.184817,4.544227,4.787516,4.700896,4.761992,7.0,3.56468,...,3.391857,3.758561,4.475235,5.562122,4.434518,3.541817,3.394817,4.5838,3.631817,4.755576
박찬수,3.789064,3.122376,3.312604,5.176974,3.517096,3.833478,3.038609,4.150228,3.56468,7.0,...,3.580611,3.081848,3.840309,3.871927,3.140272,5.029974,5.386974,3.501099,4.519974,3.409418


## 2015년

In [221]:
matrixs[1]

,박규상,정상명,김창봉,김경찬,유승덕,김병철,양홍석,김송규,하창룡,김경식,...,신인식,송혁,김재중,이현식,권인섭,장광수,박찬수,조경순,이인형,최명재
박규상,7.0,3.188943,3.398943,4.118706,4.437211,4.356751,4.017505,5.102342,4.507018,4.408743,...,4.717018,4.38745,4.696815,4.748579,4.041803,4.744279,3.568806,5.249751,3.488943,4.725317
정상명,3.188943,7.0,5.233,3.907074,2.861096,2.968832,3.147839,3.123552,3.215692,4.3702,...,3.214783,2.988091,3.527819,3.560449,3.549941,3.414716,4.876974,3.338427,4.549231,3.620574
김창봉,3.398943,5.233,7.0,3.760074,3.071096,2.968832,2.910839,3.186552,3.662692,3.9232,...,3.871783,3.345091,3.437819,3.617449,3.192941,3.477716,5.029974,3.401427,5.416231,3.320574
김경찬,4.118706,3.907074,3.760074,7.0,3.870896,3.598595,3.120603,3.753316,4.145455,4.666974,...,4.144546,3.617855,4.237619,3.890213,4.559741,4.254479,3.9631,4.86819,3.550074,4.550338
유승덕,4.437211,2.861096,3.071096,3.870896,7.0,4.738931,4.13131,4.437568,3.924108,5.027556,...,4.134192,4.435981,4.081409,4.641191,4.081631,4.640955,3.320996,4.736856,3.371096,4.219339
김병철,4.356751,2.968832,2.968832,3.598595,4.738931,7.0,4.057421,4.680387,4.260544,4.188632,...,4.262075,4.604807,3.819034,4.730211,4.030708,4.727719,3.138695,4.33827,2.968832,4.349506
양홍석,4.017505,3.147839,2.910839,3.120603,4.13131,4.057421,7.0,4.204408,3.400373,3.653301,...,3.610374,3.757393,3.563645,4.567342,3.410649,4.414339,3.227703,3.757401,2.910839,3.40032
김송규,5.102342,3.123552,3.186552,3.753316,4.437568,4.680387,4.204408,7.0,4.287726,4.553352,...,4.497726,4.872081,4.547257,5.043245,4.186166,5.187233,3.503416,4.804619,3.486552,4.212245
하창룡,4.507018,3.215692,3.662692,4.145455,3.924108,4.260544,3.400373,4.287726,7.0,4.135492,...,6.19,4.48527,4.368886,4.861587,4.371989,4.497726,3.685555,4.814807,3.152692,4.761992
김경식,4.408743,4.3702,3.9232,4.666974,5.027556,4.188632,3.653301,4.553352,4.135492,7.0,...,4.344583,4.417891,4.794279,4.690249,4.396401,4.844516,4.327074,5.068227,4.0132,4.840374


## 2016년

In [222]:
matrixs[2]

,최명재,송혁,이정화,김재중,신재국,정태암,이문수,박동현,권인섭,이어룡,...,김성원,양홍석,정재중,한여선,조경순,김창봉,하창룡,최범진,이지원,배영훈
최명재,7.0,4.227349,4.656751,4.804424,3.620574,4.41764,3.891895,4.487516,4.22411,3.239367,...,4.921193,3.399887,4.921193,4.330374,5.028539,3.320574,4.461992,4.330374,3.320574,4.614642
송혁,4.227349,7.0,4.393813,4.46966,3.198091,3.6558,3.790129,4.736945,4.561977,3.374152,...,3.868492,3.756957,4.373414,3.697891,4.633,3.345091,4.78527,3.907891,2.988091,4.625477
이정화,4.656751,4.393813,7.0,4.643678,3.271635,4.634227,4.648783,4.680176,4.141467,3.176447,...,4.651735,3.399902,5.728735,3.981435,4.745329,3.718635,4.638435,4.791435,3.571635,4.316806
김재중,4.804424,4.46966,4.643678,7.0,3.737819,4.030167,4.24199,4.087321,4.739751,2.865952,...,4.510508,3.563212,4.720508,4.843281,4.552479,3.437819,4.068886,4.794279,3.737819,4.449338
신재국,3.620574,3.198091,3.271635,3.737819,7.0,3.136951,3.371479,3.23219,3.339941,2.601812,...,3.89547,3.057406,3.32847,4.270074,3.401427,5.094805,3.215692,3.8602,5.8,3.449091
정태암,4.41764,3.6558,4.634227,4.030167,3.136951,7.0,4.9028,3.91154,3.914178,2.969339,...,5.129751,3.014952,4.802342,3.846751,4.004536,3.136951,3.888576,4.146751,3.436951,3.747056
이문수,3.891895,3.790129,4.648783,4.24199,3.371479,4.9028,7.0,3.927279,3.735576,2.806252,...,5.355735,3.835341,4.610298,4.234279,4.12223,3.224479,3.914178,4.266941,3.971479,4.247821
박동현,4.487516,4.736945,4.680176,4.087321,3.23219,3.91154,3.927279,7.0,4.601391,3.583983,...,4.126793,3.546925,4.632631,3.73199,4.482,3.67919,5.540848,3.94199,3.23219,4.620234
권인섭,4.22411,4.561977,4.141467,4.739751,3.339941,3.914178,3.735576,4.601391,7.0,3.063386,...,4.021782,3.557215,4.231782,3.788403,4.044551,3.192941,4.371989,4.396401,3.339941,4.386679
이어룡,3.239367,3.374152,3.176447,2.865952,2.601812,2.969339,2.806252,3.583983,3.063386,7.0,...,3.179357,3.618531,3.176363,2.693966,3.386776,3.405812,3.584054,2.729261,2.601812,3.43908


## 2017년

In [223]:
matrixs[3]

,이어룡,박동현,김재중,최범진,정태영,김창봉,조경순,김성원,정태암,김범철,...,홍대한,최명재,정재중,박찬수,박규상,신재국,한여선,권인섭,나재철,양홍석
이어룡,7.0,3.581246,2.863579,3.520485,3.059448,2.746376,3.38394,3.176824,2.966824,3.123906,...,3.383969,3.236825,3.173825,2.91624,3.348905,3.256376,2.691625,3.060906,4.477236,3.619061
박동현,3.581246,7.0,4.17493,4.70666,4.491435,4.0132,4.529805,4.576,4.366,5.36966,...,4.507018,4.933,5.08,4.483063,4.220211,3.8602,3.824227,5.06966,3.68696,3.546754
김재중,2.863579,4.17493,7.0,4.84827,4.788395,3.32847,4.849002,4.47493,3.96493,4.54827,...,4.741782,4.77493,4.68493,3.57837,4.589825,3.62847,5.118662,4.54827,3.610842,3.563114
최범진,3.520485,4.70666,4.84827,7.0,4.118095,3.7132,4.666465,4.85966,4.64966,4.933,...,4.643678,4.85966,5.51666,3.9631,4.266871,4.3702,4.129889,4.723,3.30459,3.263094
정태영,3.059448,4.491435,4.788395,4.118095,7.0,3.481635,4.290387,4.241535,3.521535,4.887097,...,4.357619,4.281435,3.981435,3.828509,4.480249,3.781635,4.866751,4.118095,3.864842,4.015792
김창봉,2.746376,4.0132,3.32847,3.7132,3.481635,7.0,3.520005,4.2802,3.5032,4.0702,...,3.644218,3.7132,3.7132,5.386974,3.420411,5.094805,3.581301,3.6502,3.28406,3.057294
조경순,3.38394,4.529805,4.849002,4.666465,4.290387,3.520005,7.0,4.829805,4.109805,4.576465,...,5.767619,5.129805,4.829805,3.689868,4.969355,3.520005,4.412342,4.156465,3.832636,3.609778
김성원,3.176824,4.576,4.47493,4.85966,4.241535,4.2802,4.829805,7.0,5.323,5.36966,...,4.654018,5.023,5.023,4.450063,5.240211,3.9232,4.544227,4.34966,4.039214,3.881824
정태암,2.966824,4.366,3.96493,4.64966,3.521535,3.5032,4.109805,5.323,7.0,4.50266,...,4.087018,4.66,5.023,3.673063,4.220211,3.5032,3.824227,4.20266,3.172214,3.014824
김범철,3.123906,5.36966,4.54827,4.933,4.887097,4.0702,4.576465,5.36966,4.50266,7.0,...,4.910678,4.76966,5.06966,4.540063,4.566871,3.9232,4.334227,4.87,4.03362,4.367094


## 2018년

In [224]:
matrixs[4]

,정태암,박성준,이환목,김재중,김병철,이순남,정연규,김상원,오익근,진승욱,...,최명재,이동훈,나재철,이득원,신인식,송혁,김호중,김범철,신재국,이정화
정태암,7.0,4.116695,3.609201,3.89866,3.5032,4.723,4.723,4.87,4.20266,4.34966,...,4.503515,4.023798,3.14948,4.34966,4.209515,3.999515,4.723,4.346175,3.5032,4.866515
박성준,4.116695,7.0,4.270612,4.98493,3.616895,5.346695,5.136695,5.346695,5.240355,5.483355,...,4.844227,4.3243,3.959827,5.483355,4.75827,4.33827,5.193695,4.89493,3.526895,4.844227
이환목,3.609201,4.270612,7.0,4.667469,3.269301,4.239201,4.029201,3.939201,4.142226,4.442226,...,4.031254,3.61236,2.957438,4.442226,3.941254,3.521254,4.239201,4.444279,3.479301,4.631254
김재중,3.89866,4.98493,4.667469,7.0,3.0522,4.61866,4.61866,4.61866,4.182,4.782,...,4.712291,4.161407,3.590739,4.782,4.324176,4.534176,4.31866,4.487516,3.5622,4.622291
김병철,3.5032,3.616895,3.269301,3.0522,7.0,4.0132,3.8032,4.0132,4.0132,4.1602,...,3.556715,2.950998,2.904326,4.1602,4.003715,3.136715,4.3132,3.703715,5.59,3.346715
이순남,4.723,5.346695,4.239201,4.61866,4.0132,7.0,5.59,6.043,5.06966,5.57966,...,4.776515,4.023798,3.904775,5.57966,4.776515,4.503515,5.743,4.913175,3.9232,5.133515
정연규,4.723,5.136695,4.029201,4.61866,3.8032,5.59,7.0,5.233,5.06966,5.36966,...,4.866515,4.323798,3.445196,5.36966,5.076515,5.013515,5.233,4.613175,3.7132,5.643515
김상원,4.87,5.346695,3.939201,4.61866,4.0132,6.043,5.233,7.0,5.06966,5.57966,...,5.133515,4.023798,4.204775,5.57966,4.776515,4.356515,5.743,4.913175,3.9232,4.776515
오익근,4.20266,5.240355,4.142226,4.182,4.0132,5.06966,5.06966,5.06966,7.0,5.443,...,4.613175,3.860458,3.563886,5.443,5.123175,4.703175,5.21666,5.286515,3.9232,4.613175
진승욱,4.34966,5.483355,4.442226,4.782,4.1602,5.57966,5.36966,5.57966,5.443,7.0,...,4.913175,4.517458,3.938856,6.19,4.970175,4.403175,5.27966,5.643515,4.3702,4.913175


## 2019년

In [225]:
matrixs[5]

,최명재,진승욱,권택현,이재우,정연우,문병식,이환목,신재국,박동현,정태암,...,홍대한,길기모,이정화,송혁,이순남,신인식,임민수,김상원,이성근,오익근
최명재,7.0,4.809423,4.395309,4.644518,4.944518,4.644518,4.346348,3.724718,5.89,4.090619,...,4.804587,4.644518,5.17,4.976751,4.739751,5.076515,4.791518,4.502342,4.509423,4.509423
진승욱,4.809423,7.0,5.041288,5.177853,5.123175,4.970175,4.098632,4.213715,4.794279,4.71266,...,4.98311,5.019177,4.809423,4.55966,5.386411,5.12666,4.613175,5.213175,5.172122,5.59
권택현,4.395309,5.041288,7.0,4.604436,4.604436,4.304436,3.607078,3.084636,4.395407,4.457532,...,4.608964,4.304436,4.815309,4.395604,4.905016,4.605505,4.094436,4.694822,4.14256,4.441288
이재우,4.644518,5.177853,4.604436,7.0,6.4,5.743,4.19509,3.9232,4.423,5.023,...,4.697516,5.143,4.644518,4.403507,5.50064,4.622741,5.533,5.249751,5.237047,4.667853
정연우,4.944518,5.123175,4.604436,6.4,7.0,5.743,4.194577,3.9232,4.423,4.921193,...,4.695631,5.143,4.644518,4.403507,5.50064,4.613507,5.533,5.222342,5.230508,4.613175
문병식,4.644518,4.970175,4.304436,5.743,5.743,7.0,4.194577,4.0702,4.423,4.621193,...,5.442631,5.59,4.644518,4.403507,5.20064,4.760507,5.533,4.922342,5.677508,5.060175
이환목,4.346348,4.098632,3.607078,4.19509,4.194577,4.194577,7.0,3.434677,4.346902,3.77509,...,3.923083,4.017566,4.436348,3.628023,4.497717,3.901023,4.131577,3.897183,4.405768,3.889153
신재국,3.724718,4.213715,3.084636,3.9232,3.9232,4.0702,3.434677,7.0,3.4132,3.611393,...,3.622831,4.0702,3.424718,3.393707,3.68084,3.750707,3.7132,3.702542,3.721048,3.913715
박동현,5.89,4.794279,4.395407,4.423,4.423,4.423,4.346902,3.4132,7.0,4.077518,...,4.807018,4.423,5.17,5.013515,4.712342,5.431193,5.08,4.48064,4.3173,4.494279
정태암,4.090619,4.71266,4.457532,5.023,4.921193,4.621193,3.77509,3.611393,4.077518,7.0,...,4.130516,4.174193,4.810619,4.055741,4.802342,4.265741,4.411193,5.186751,4.22411,4.207953


## 2020년

In [226]:
matrixs[6]

,김병철,오익근,이정화,진승욱,길기모,정재중,송혁,이재우,신인식,조경순,...,정연우,신재범,홍대한,이어룡,이지원,홍종국,양홍석,조홍희,강준규,문병식
김병철,7.0,3.479353,3.122721,3.7132,3.7132,3.779721,2.88677,3.364048,4.270376,3.184817,...,3.47084,3.320574,3.252275,3.249592,5.233,3.385322,3.202686,5.233,3.7132,3.332721
오익근,3.479353,7.0,3.787987,4.601148,4.181915,3.715957,3.832479,3.922267,4.001233,4.509501,...,3.916556,4.074885,4.469252,3.649512,3.536353,3.921082,3.765573,3.116353,3.913883,4.226593
이정화,3.122721,3.787987,7.0,4.741782,4.552521,5.443,4.462763,4.672763,5.012342,4.718597,...,4.577764,4.776515,4.817587,3.070535,3.632721,4.644518,3.539159,3.122721,4.909521,4.933
진승욱,3.7132,4.601148,4.741782,7.0,4.875662,4.450167,4.340411,5.0373,4.924424,4.757257,...,5.059002,4.784047,4.98311,3.117155,4.0702,4.786411,4.065491,3.3502,4.67966,4.651782
길기모,3.7132,4.181915,4.552521,4.875662,7.0,4.252521,4.16357,4.793848,4.533176,4.914617,...,4.90064,4.597374,5.129075,2.980462,4.3702,4.815122,3.651148,3.6502,4.843,5.062521
정재중,3.779721,3.715957,5.443,4.450167,4.252521,7.0,4.237619,4.99064,5.34764,4.417891,...,4.883848,5.039751,4.514807,3.677111,3.632721,4.944518,4.049159,3.122721,4.852521,5.233
송혁,2.88677,3.832479,4.462763,4.340411,4.16357,4.237619,7.0,4.17745,5.131193,4.423763,...,4.124227,4.949342,4.687374,3.289588,3.24377,4.163848,3.959163,3.03377,4.40657,4.252763
이재우,3.364048,3.922267,4.672763,5.0373,4.793848,4.99064,4.17745,7.0,4.613507,4.981473,...,5.732741,5.154518,4.709945,3.230094,3.874048,5.586515,3.755155,3.154048,5.093848,5.839763
신인식,4.270376,4.001233,5.012342,4.924424,4.533176,5.34764,5.131193,4.613507,7.0,4.932245,...,4.552521,5.076515,5.043176,4.084551,3.523376,4.59745,4.406162,2.893376,4.833176,4.712342
조경순,3.184817,4.509501,4.718597,4.757257,4.914617,4.417891,4.423763,4.981473,4.932245,7.0,...,4.917536,4.420032,5.556396,3.427889,3.394817,4.622506,3.749227,2.764817,4.404617,5.285597


## 2021년

In [227]:
matrixs[7]

,이어룡,양홍석,강준규,김범철,김병철,이성영,원윤희,진승욱,이정훈,박정환,...,김호중,김창수,박성준,오익근,나유석,길기모,권택현,김수창,홍종국,송혁
이어룡,7.0,4.412007,3.124247,3.186544,2.589662,3.064265,2.933762,3.114343,2.788917,2.824211,...,3.011335,2.276762,3.278547,3.647128,2.914682,2.977605,3.070862,3.437208,3.227241,3.286841
양홍석,4.412007,7.0,3.34997,4.210265,2.543403,3.227523,2.990403,3.916232,3.026935,3.188965,...,3.616211,2.543403,3.753056,3.973395,3.140053,3.501879,3.54348,3.139982,3.349988,3.746917
강준규,3.124247,3.34997,7.0,4.264379,3.385322,4.99064,3.175322,4.548891,4.515122,4.605122,...,5.624853,3.385322,4.552409,3.857739,4.99659,4.523507,4.274177,4.861735,5.030469,4.020233
김범철,3.186544,4.210265,4.264379,7.0,3.345091,4.165685,2.988091,5.334579,4.003553,3.793553,...,4.638539,3.345091,4.711818,4.366057,4.106909,4.380011,4.415906,4.428989,4.294732,4.664047
김병철,2.589662,2.543403,3.385322,3.345091,7.0,3.703951,4.674805,3.704069,3.8602,3.7132,...,3.670587,5.031805,3.420084,3.567657,2.960584,3.471427,3.143718,3.442007,3.26965,2.935739
이성영,3.064265,3.227523,4.99064,4.165685,3.703951,7.0,3.436951,4.351467,4.923751,4.356751,...,4.702002,4.003951,4.196413,3.920038,4.189153,4.39064,3.917484,4.53445,4.840374,3.775429
원윤희,2.933762,2.990403,3.175322,2.988091,4.674805,3.436951,7.0,3.047069,3.8032,3.8032,...,3.160587,5.533,2.910084,2.910657,2.960584,3.114427,2.843718,3.085007,3.05965,2.641739
진승욱,3.114343,3.916232,4.548891,5.334579,3.704069,4.351467,3.047069,7.0,4.209531,4.062531,...,5.014587,3.404069,5.296963,4.548095,4.255257,4.775838,5.01282,4.760411,4.704279,4.265095
이정훈,2.788917,3.026935,4.515122,4.003553,3.8602,4.923751,3.8032,4.209531,7.0,5.233,...,4.176049,4.1602,4.039884,4.300768,4.090384,4.991227,3.763518,4.361807,4.39945,3.561539
박정환,2.824211,3.188965,4.605122,3.793553,3.7132,4.356751,3.8032,4.062531,5.233,7.0,...,4.176049,3.8032,4.186884,3.394189,4.327384,4.844227,3.910518,4.514807,4.48945,3.351539


## 2022년

In [228]:
matrixs[8]

,박정환,강준규,최근영,이순남,김병철,이정화,최광철,이재우,길기모,김성호,...,김범철,이창세,신재범,김창수,송종원,박성준,김광혁,김경렬,원윤희,김호중
박정환,7.0,4.48945,4.094973,4.415819,3.7132,3.916909,4.933,4.420689,4.807018,3.7132,...,3.719277,4.0132,4.443245,3.8032,4.933,4.152505,4.723,4.723,3.8032,4.09268
강준규,4.48945,7.0,4.898652,4.96436,3.26965,4.776869,4.78945,4.58809,4.554518,3.26965,...,4.190025,3.26965,5.322123,3.26965,5.29945,4.519921,4.78945,4.78945,3.05965,5.291178
최근영,4.094973,4.898652,7.0,4.8273,3.136511,4.67966,4.492971,4.243012,4.183496,3.136511,...,4.307124,2.989511,5.266465,3.346511,4.702971,4.153589,4.912971,4.765971,2.989511,4.750834
이순남,4.415819,4.96436,4.8273,7.0,3.496019,4.583848,4.715819,4.919355,4.355329,3.196019,...,4.508761,3.496019,5.171807,3.196019,5.225819,5.127549,4.715819,5.162819,2.986019,4.638989
김병철,3.7132,3.26965,3.136511,3.496019,7.0,2.997109,4.0702,3.143889,3.434218,5.241805,...,3.270815,5.533,3.223445,5.031805,3.8602,3.385705,3.6502,3.5032,4.674805,3.587218
이정화,3.916909,4.776869,4.67966,4.583848,2.997109,7.0,4.006909,4.700743,4.463348,2.787109,...,4.503939,3.654109,4.572081,3.207109,4.516909,4.615713,4.426909,4.426909,3.654109,4.900008
최광철,4.933,4.78945,4.492971,4.715819,4.0702,4.006909,7.0,4.363689,4.354018,4.1602,...,3.907275,3.7132,4.743245,3.6502,5.68,4.305505,5.68,5.323,3.5032,4.343678
이재우,4.420689,4.58809,4.243012,4.919355,3.143889,4.700743,4.363689,7.0,4.885196,3.143889,...,4.783074,3.143889,4.601762,3.143889,4.573689,5.59216,4.363689,4.663689,2.933889,4.721504
길기모,4.807018,4.554518,4.183496,4.355329,3.434218,4.463348,4.354018,4.885196,7.0,3.434218,...,4.29389,3.587218,4.403474,3.434218,4.564018,4.51907,4.354018,4.207018,3.077218,5.028662
김성호,3.7132,3.26965,3.136511,3.196019,5.241805,2.787109,4.1602,3.143889,3.434218,7.0,...,3.060815,5.394805,3.223445,5.68,4.1602,3.085705,3.9502,3.8032,5.233,3.287218


## 2023년

In [229]:
matrixs[9]

,김봉진,김성호,양홍석,강준규,최근영,권택현,신재범,정회민,이순남,박석원,...,윤태림,이정훈,김수창,김상원,이어룡,홍종국,길기모,김범철,김성균,한승희
김봉진,7.0,3.5032,3.438886,4.773638,4.747942,4.773412,5.090685,4.087018,5.156973,4.538579,...,4.513,4.52657,4.529258,5.166311,2.912074,4.72937,4.197726,4.262874,5.213507,3.7132
김성호,3.5032,7.0,2.542325,3.252275,3.117943,2.833572,3.213885,3.377218,3.190173,3.674679,...,3.8602,3.450661,3.365497,3.198797,2.484625,3.208739,3.424926,3.053414,3.685322,5.331805
양홍석,3.438886,2.542325,7.0,3.348915,3.3023,3.542505,3.348934,2.806454,3.648951,3.138926,...,2.913534,3.025874,3.138931,3.648944,4.411096,3.348937,3.500809,4.209322,3.795892,2.439425
강준규,4.773638,3.252275,3.348915,7.0,4.875918,4.265596,5.322123,4.299081,4.965576,4.861735,...,4.262075,4.450374,4.82657,4.985235,3.122224,5.364726,4.50745,4.183196,5.00557,3.252275
최근영,4.747942,3.117943,3.3023,4.875918,7.0,4.315814,5.21866,3.840423,4.720508,4.534321,...,3.964403,4.151259,4.547298,4.660121,2.749114,4.630134,4.159571,4.304279,4.544797,3.327943
권택현,4.773412,2.833572,3.542505,4.265596,4.315814,7.0,4.267012,3.697842,4.868429,4.056405,...,3.696372,3.754552,4.056794,4.867803,3.069206,4.267274,4.265009,4.417002,4.563897,3.043572
신재범,5.090685,3.213885,3.348934,5.322123,5.21866,4.267012,7.0,4.056869,5.171807,4.890387,...,4.223685,4.37614,4.99064,5.223751,3.122317,5.20064,4.393793,4.190756,4.955329,3.213885
정회민,4.087018,3.377218,2.806454,4.299081,3.840423,3.697842,4.056869,7.0,4.016414,4.594587,...,4.597018,4.162763,4.065631,4.030834,2.852172,4.194579,3.872751,3.299769,4.424227,3.077218
이순남,5.156973,3.190173,3.648951,4.965576,4.720508,4.868429,5.171807,4.016414,7.0,4.783793,...,4.199973,4.341063,4.802231,5.85445,3.225308,5.043176,4.350233,4.499485,5.227891,3.190173
박석원,4.538579,3.674679,3.138926,4.861735,4.534321,4.056405,4.890387,4.594587,4.783793,7.0,...,4.384479,4.33645,5.091518,4.814807,3.122278,5.161735,4.361807,3.914423,5.361992,3.164679


중요인물 행만 뽑아서 네트워크 만들면 될듯..? 아마도..?



# 가중치 행렬 스케일 조정

In [254]:
# 인물 가중치행렬
col = []
row = []
matrixs = []
dataFrames = [df_2014, df_2015, df_2016, df_2017, df_2018, df_2019, df_2020, df_2021, df_2022, df_2023]

for dataframe in dataFrames:
  for i in range(len(dataframe)):
      col.append(dataframe.loc[i, '이름'])
      row.append(dataframe.loc[i, '이름'])
  weight_matrix = pd.DataFrame(columns=col, index=row)
  matrixs.append(weight_matrix)
  col=[]
  row=[]

In [255]:
#가중치 계산
for t, dataframe in enumerate(dataFrames):
  for i in range(0, len(dataframe)):
    for j in range(0, len(dataframe)):
      matrixs[t].iloc[i, j] = weight_sum(dataframe.iloc[i], dataframe.iloc[j])

In [256]:
matrixs[6]

,김병철,오익근,이정화,진승욱,길기모,정재중,송혁,이재우,신인식,조경순,...,정연우,신재범,홍대한,이어룡,이지원,홍종국,양홍석,조홍희,강준규,문병식
김병철,7.0,3.479353,3.122721,3.7132,3.7132,3.779721,2.88677,3.364048,4.270376,3.184817,...,3.47084,3.320574,3.252275,3.249592,5.233,3.385322,3.202686,5.233,3.7132,3.332721
오익근,3.479353,7.0,3.787987,4.601148,4.181915,3.715957,3.832479,3.922267,4.001233,4.509501,...,3.916556,4.074885,4.469252,3.649512,3.536353,3.921082,3.765573,3.116353,3.913883,4.226593
이정화,3.122721,3.787987,7.0,4.741782,4.552521,5.443,4.462763,4.672763,5.012342,4.718597,...,4.577764,4.776515,4.817587,3.070535,3.632721,4.644518,3.539159,3.122721,4.909521,4.933
진승욱,3.7132,4.601148,4.741782,7.0,4.875662,4.450167,4.340411,5.0373,4.924424,4.757257,...,5.059002,4.784047,4.98311,3.117155,4.0702,4.786411,4.065491,3.3502,4.67966,4.651782
길기모,3.7132,4.181915,4.552521,4.875662,7.0,4.252521,4.16357,4.793848,4.533176,4.914617,...,4.90064,4.597374,5.129075,2.980462,4.3702,4.815122,3.651148,3.6502,4.843,5.062521
정재중,3.779721,3.715957,5.443,4.450167,4.252521,7.0,4.237619,4.99064,5.34764,4.417891,...,4.883848,5.039751,4.514807,3.677111,3.632721,4.944518,4.049159,3.122721,4.852521,5.233
송혁,2.88677,3.832479,4.462763,4.340411,4.16357,4.237619,7.0,4.17745,5.131193,4.423763,...,4.124227,4.949342,4.687374,3.289588,3.24377,4.163848,3.959163,3.03377,4.40657,4.252763
이재우,3.364048,3.922267,4.672763,5.0373,4.793848,4.99064,4.17745,7.0,4.613507,4.981473,...,5.732741,5.154518,4.709945,3.230094,3.874048,5.586515,3.755155,3.154048,5.093848,5.839763
신인식,4.270376,4.001233,5.012342,4.924424,4.533176,5.34764,5.131193,4.613507,7.0,4.932245,...,4.552521,5.076515,5.043176,4.084551,3.523376,4.59745,4.406162,2.893376,4.833176,4.712342
조경순,3.184817,4.509501,4.718597,4.757257,4.914617,4.417891,4.423763,4.981473,4.932245,7.0,...,4.917536,4.420032,5.556396,3.427889,3.394817,4.622506,3.749227,2.764817,4.404617,5.285597


In [259]:
#가중치 계산
for t, dataframe in enumerate(dataFrames):
  for i in range(0, len(dataframe)):
    for j in range(0, len(dataframe)):
      matrixs[t].iloc[i, j] = math.log2(1+ weight_sum(dataframe.iloc[i], dataframe.iloc[j])**3)

In [260]:
matrixs[6]

,김병철,오익근,이정화,진승욱,길기모,정재중,송혁,이재우,신인식,조경순,...,정연우,신재범,홍대한,이어룡,이지원,홍종국,양홍석,조홍희,강준규,문병식
김병철,8.426265,5.430308,4.975028,5.705897,5.705897,5.781312,4.647126,5.288001,6.301498,5.057614,...,5.419952,5.233173,5.145688,5.142217,7.172947,5.314593,5.081108,7.172947,5.705897,5.248559
오익근,5.430308,8.426265,5.790596,6.620717,6.212084,5.709047,5.840238,5.938776,6.023681,6.534551,...,5.932572,6.101444,6.496172,5.63249,5.499045,5.93749,5.765378,4.966474,5.929667,6.257468
이정화,4.975028,5.790596,8.426265,6.749756,6.575207,7.342125,6.489953,6.686899,6.987866,6.728741,...,6.598891,6.781054,6.817778,4.904463,5.612936,6.660915,5.502402,4.975028,6.898887,6.919364
진승욱,5.705897,6.620717,6.749756,8.426265,6.869188,6.477856,6.371031,7.009195,6.911896,6.763728,...,7.027658,6.787811,6.962752,4.967552,6.096538,6.78993,6.091601,5.270608,6.693221,6.667612
길기모,5.705897,6.212084,6.575207,6.869188,8.426265,6.283594,6.193314,6.796589,6.556971,6.903339,...,6.891117,6.617202,7.086748,4.780095,6.40028,6.815582,5.63439,5.633289,6.840348,7.030645
정재중,5.781312,5.709047,7.342125,6.477856,6.283594,8.426265,6.268597,6.969235,7.26611,6.446709,...,6.876387,7.011285,6.539586,5.664447,5.612936,6.929375,6.074436,4.975028,6.848775,7.172947
송혁,4.647126,5.840238,6.489953,6.371031,6.193314,6.268597,8.426265,6.207523,7.088523,6.452392,...,6.152792,6.933561,6.700281,5.19369,5.134676,6.193599,5.978648,4.8541,6.435731,6.283837
이재우,5.288001,5.938776,6.686899,7.009195,6.796589,6.969235,6.207523,8.426265,6.632209,6.961341,...,7.565313,7.108009,6.720873,5.116914,5.886127,7.454097,5.753609,5.016876,7.057142,7.644956
신인식,6.301498,6.023681,6.987866,6.911896,6.556971,7.26611,7.088523,6.632209,8.426265,6.918708,...,6.575207,7.042501,7.014202,6.11155,5.483488,6.617273,6.435335,4.656626,6.831637,6.723054
조경순,5.057614,6.534551,6.728741,6.763728,6.903339,6.446709,6.452392,6.961341,6.918708,8.426265,...,6.905889,6.448782,7.430834,5.367341,5.326412,6.640558,5.746897,4.468247,6.433834,7.215936


In [261]:
#가중치 계산
for t, dataframe in enumerate(dataFrames):
  for i in range(0, len(dataframe)):
    for j in range(0, len(dataframe)):
      matrixs[t].iloc[i, j] = math.sqrt(weight_sum(dataframe.iloc[i], dataframe.iloc[j])**3)

In [262]:
matrixs[6]

,김병철,오익근,이정화,진승욱,길기모,정재중,송혁,이재우,신인식,조경순,...,정연우,신재범,홍대한,이어룡,이지원,홍종국,양홍석,조홍희,강준규,문병식
김병철,18.520259,6.490046,5.51823,7.155212,7.155212,7.348347,4.904765,6.170115,8.824686,5.683641,...,6.466241,6.050897,5.865173,5.857917,11.970881,6.228736,5.731542,11.970881,7.155212,6.08413
오익근,6.490046,18.520259,7.372465,9.869596,8.551904,7.163182,7.502736,7.767936,8.0037,9.576191,...,7.750977,8.225703,9.44827,6.971918,6.65018,7.764419,7.307126,5.501359,7.743045,8.689318
이정화,5.51823,7.372465,18.520259,10.32553,9.713549,12.698648,9.4277,10.100913,11.221762,10.249894,...,9.794452,10.439189,10.574124,5.38048,6.923857,10.009468,6.658097,5.51823,10.878241,10.95637
진승욱,7.155212,9.869596,10.32553,18.520259,10.765901,9.387814,9.042661,11.305681,10.927812,10.376118,...,11.378822,10.46389,11.123736,5.503483,8.211521,10.471648,8.197276,6.132056,10.123285,10.032958
길기모,7.155212,8.551904,9.713549,10.765901,18.520259,8.769397,8.495692,10.496061,9.651703,10.895181,...,10.848737,9.857455,11.616051,5.145474,9.135913,10.566008,6.976604,6.973888,10.657902,11.390696
정재중,7.348347,7.163182,12.698648,9.387814,8.769397,18.520259,8.723341,11.14896,12.366398,9.285869,...,10.793025,11.313935,9.593097,7.051151,6.923857,10.994765,8.147929,5.51823,10.689346,11.970881
송혁,4.904765,7.502736,9.4277,9.042661,8.495692,8.723341,18.520259,8.538212,11.62325,9.304389,...,8.37556,11.010859,10.148327,5.966399,5.842181,8.496543,7.877804,5.284135,9.250196,8.770145
이재우,6.170115,7.767936,10.100913,11.305681,10.496061,11.14896,8.538212,18.520259,9.909387,11.118257,...,13.725985,11.702593,10.221715,5.805274,7.625134,13.204182,7.276823,5.601475,11.496588,14.112139
신인식,8.824686,8.0037,11.221762,10.927812,9.651703,12.366398,11.62325,9.909387,18.520259,10.953855,...,9.713549,11.437959,11.32547,8.25499,6.61361,9.857699,9.248914,4.921612,10.62549,10.229519
조경순,5.683641,9.576191,10.249894,10.376118,10.895181,9.285869,9.304389,11.118257,10.953855,18.520259,...,10.904892,9.292619,13.097541,6.346585,6.254959,9.938394,7.259597,4.597264,9.244047,12.151813


In [257]:
#가중치 계산
for t, dataframe in enumerate(dataFrames):
  for i in range(0, len(dataframe)):
    for j in range(0, len(dataframe)):
      matrixs[t].iloc[i, j] = weight_sum(dataframe.iloc[i], dataframe.iloc[j])**2

In [258]:
matrixs[6]

,김병철,오익근,이정화,진승욱,길기모,정재중,송혁,이재우,신인식,조경순,...,정연우,신재범,홍대한,이어룡,이지원,홍종국,양홍석,조홍희,강준규,문병식
김병철,49.0,12.105898,9.751387,13.787854,13.787854,14.286292,8.333439,11.316818,18.236115,10.143057,...,12.04673,11.026213,10.57729,10.559848,27.384289,11.460404,10.257195,27.384289,13.787854,11.10703
오익근,12.105898,49.0,14.348845,21.170566,17.488414,13.808336,14.687895,15.384175,16.009867,20.335602,...,15.339407,16.604688,19.974214,13.31894,12.505793,15.374887,14.17954,9.711656,15.318481,17.86409
이정화,9.751387,14.348845,49.0,22.484495,20.725448,29.626249,19.916254,21.834715,25.123574,22.265159,...,20.955927,22.815099,23.209147,9.428184,13.196663,21.571549,12.525647,9.751387,24.103397,24.334489
진승욱,13.787854,21.170566,22.484495,49.0,23.77208,19.803988,18.839167,25.374391,24.249956,22.631493,...,25.593503,22.887107,24.831383,9.716656,16.566528,22.909734,16.52822,11.22384,21.899218,21.639074
길기모,13.787854,17.488414,20.725448,23.77208,49.0,18.083936,17.335312,22.980977,20.549689,24.153456,...,24.016272,21.13585,26.307406,8.883153,19.098648,23.185398,13.330879,13.32396,23.454649,25.62912
정재중,14.286292,13.808336,29.626249,19.803988,18.083936,49.0,17.957413,24.906487,28.597253,19.517764,...,23.85197,25.399094,20.383486,13.521142,13.196663,24.448259,16.395689,9.751387,23.546961,27.384289
송혁,8.333439,14.687895,19.916254,18.839167,17.335312,17.957413,49.0,17.451091,26.329146,19.569683,...,17.009249,24.495988,21.971477,10.82139,10.522042,17.337629,15.674976,9.203758,19.417856,18.085994
이재우,11.316818,15.384175,21.834715,25.374391,22.980977,24.906487,17.451091,49.0,21.284448,24.815075,...,32.864323,26.569057,22.183582,10.433507,15.008247,31.209154,14.101191,9.948018,25.947286,34.102833
신인식,18.236115,16.009867,25.123574,24.249956,20.549689,28.597253,26.329146,21.284448,49.0,24.327044,...,20.725448,25.771008,25.433629,16.683561,12.414182,21.13655,19.414265,8.371627,23.359595,22.206169
조경순,10.143057,20.335602,22.265159,22.631493,24.153456,19.517764,19.569683,24.815075,24.327044,49.0,...,24.182165,19.536684,30.873535,11.750421,11.524779,21.36756,14.0567,7.644211,19.400647,27.937538




---
너무 가중치 차이가 안나서 네트워크 그리기 힘들면

alpha 값을 조정하거나
스케일을 조절해야될듯.

그나마 가중치 차이 많이 볼 수 있는 스케일 조정..?
1. 가중치 제곱
2. 루트(가중치 세제곱)

=> 그 후 threshold(임계값) 조정 